In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
import numpy as np

In [2]:
BATCH_SIZE = 64
EPOCHS = 5
INPUT_SIZE = 11
LAYER_SIZE = 20
LATENT_SIZE = 3

In [3]:
# Load the data from the file

saved = np.load("testing.npy", allow_pickle=True)

# Make a trainging and testing batch
train_data = torch.Tensor(saved[:int(len(saved)*0.9)])
test_data = torch.Tensor(saved[int(len(saved*0.9)):])
len(train_data)

90000

In [4]:
device = "cuda"

In [5]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        # Encoder
        self.l1 = nn.Linear(INPUT_SIZE, LAYER_SIZE)
        self.l2a = nn.Linear(LAYER_SIZE, LATENT_SIZE)
        self.l2b = nn.Linear(LAYER_SIZE, LATENT_SIZE)
        
        # Decoder
        self.l3 = nn.Linear(LATENT_SIZE, LAYER_SIZE)
        self.l4 = nn.Linear(LAYER_SIZE, INPUT_SIZE)
    
    # Run some data through the encoder
    def encode(self, x):
        out = F.relu(self.l1(x))
        # return the mu and the sigma
        return self.l2a(out), self.l2b(out)
    
    # The reparameterization trick, taken from https://github.com/pytorch/examples/blob/master/vae/main.py
    def reparameterize(self, mu, sigma):
        std = torch.exp(0.5*sigma)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, x):
        out = F.relu(self.l3(x))
        return torch.sigmoid(self.l4(out)) # sigmoid vs tanh
    
    def forward(self, x):
        # TODO: check the shape of x to be sure we have the right input
        mu, sigma = self.encode(x)
        z = self.reparameterize(mu, sigma)
        # The loss function needs the mu and the sigma so just return them here
        return self.decode(z), mu, sigma

In [6]:
# Reconstruction + KL divergence losses summed over all elements and batch
# Taken from https://github.com/pytorch/examples/blob/master/vae/main.py
def loss_function(recon_x, x, mu, sigma):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + sigma - mu.pow(2) - sigma.exp())
    return BCE + KLD

In [7]:
vae = VAE().to(device)
optimizer = optim.Adam(vae.parameters(), lr=0.001)

In [8]:
def train(e_count):
    vae.train()
    bs = 0
    train_loss = 0
    for i in range(0, len(train_data), BATCH_SIZE):
        batch = train_data[i:i+BATCH_SIZE].to(device)
        vae.zero_grad()
        recons, mu, sigma = vae(batch)
        loss = loss_function(recons, batch, mu, sigma)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
    print(f"Epoch number: {e_count}, loss: {train_loss/len(train_data)}")        
        
    
    

In [9]:
for i in range(15):
    train(i)

Epoch number: 0, loss: -64.5823937151591
Epoch number: 1, loss: -102.56528255750868
Epoch number: 2, loss: -114.20159628634983
Epoch number: 3, loss: -120.57638023003472
Epoch number: 4, loss: -124.45075209418403
Epoch number: 5, loss: -127.61899962565104
Epoch number: 6, loss: -131.744256241862
Epoch number: 7, loss: -135.0486755750868
Epoch number: 8, loss: -138.61743599989148
Epoch number: 9, loss: -141.48094783799914
Epoch number: 10, loss: -143.74610743815103
Epoch number: 11, loss: -145.6809415011936
Epoch number: 12, loss: -147.9759304877387
Epoch number: 13, loss: -150.16247517903645
Epoch number: 14, loss: -152.8623485921224
